In [1]:
!pip install pygame pymunk

In [48]:
import sys, random
import pygame
from pygame.locals import *
import pymunk
import pymunk.pygame_util
from pymunk.vec2d import Vec2d
import copy
import itertools
import time
import numpy as np

import matplotlib.pyplot as plt
    


class Game:
    
    def __init__(self):
        self.WIDTH = 600
        self.HEIGHT = 600
        
        self.DURATION = 15
        
        self.CAMERA_X = 300
        self.CAMERA_Y = 300
        
        self.distance = None
        self.circle_list = []
        self.spring_joint_list = []
        self.pin_joint_list = []
        self.line_list = []
    
    def getGameObjects(self):
        self.circle_list = [circle.restoreInitialState() for circle in self.circle_list]
        self.line_list = [line.restoreInitialState() for line in self.line_list]
        self.spring_joint_list = [joint.restoreInitialState(self.circle_list[joint.i1],self.circle_list[joint.i2]) for joint in self.spring_joint_list]
        self.pin_joint_list = [joint.restoreInitialState(self.circle_list[joint.i1],self.circle_list[joint.i2]) for joint in self.pin_joint_list] 
        return [self.circle_list,
        self.spring_joint_list,
        self.pin_joint_list,
        self.line_list]
        
    def updateCamera(self,circles):
        min_x = circles[np.argmin(np.array([circle.body.position.x for circle in circles]))].body.position.x
        min_y = circles[np.argmin(np.array([circle.body.position.y for circle in circles]))].body.position.y
        max_x = circles[np.argmax(np.array([circle.body.position.x for circle in circles]))].body.position.x
        max_y = circles[np.argmax(np.array([circle.body.position.y for circle in circles]))].body.position.y
        
        pos = (Vec2d(min_x,max_y)+Vec2d(max_x,min_y))/2
        
        # leftmost circle is focused
            # circle_with_min_distance = np.argmin(np.array([circle.body.position.x for circle in circles]))
            # camera = circles[circle_with_min_distance]
            # pos = camera.body.position

        return Vec2d(pos.x - self.CAMERA_X, pos.y-self.CAMERA_Y)
    
    def simulatePhysics(self):
        
        FPS = 50
        
        space = pymunk.Space()
        space.gravity = (0.0, -900.0)
        #-----------------------------------------------------------------
        lines = []
        lines.append(add_static_L(space,0,100))
        circles,spring_joints,pin_joints = createRandomMachine(space,False)
        #-----------------------------------------------------------------
        #lines = []
        #circles = []
        #joints = []
         
        maxStep = FPS * self.DURATION
        
        for currentStep in range(maxStep):
            
            for joint in spring_joints:
                joint.strain_countdown()

            space.step(1/FPS)
        
        self.distance = max([circle.body.position.x for circle in circles])-600
        self.circle_list = circles
        self.spring_joint_list = spring_joints
        self.pin_joint_list = pin_joints
        self.line_list = lines
            
    def start(self,replicate_game=False, gameObjects = [], DRAW_DEBUG = True):
    
        pygame.init()
        screen = pygame.display.set_mode((self.WIDTH, self.HEIGHT))
        myfont = pygame.font.SysFont('Comic Sans MS', 30)
        distance_marker_font = pygame.font.SysFont('Comic Sans MS', 10)
        pygame.display.set_caption("Game Simulation")
        clock = pygame.time.Clock()
        
        distance_marker = [100*(i+1) for i in range(20)]

        draw_options = pymunk.pygame_util.DrawOptions(screen)

        space = pymunk.Space()
        space.gravity = (0.0, -900.0)

        if not replicate_game:
            #-----------------------------------------------------------------
            lines = []
            lines.append(add_static_L(space,0,100))
            circles,spring_joints,pin_joints = createRandomMachine(space,False)
            #-----------------------------------------------------------------
            #lines = []
            #circles = []
            #joints = []
        else:
            
            circles = gameObjects[0]
            spring_joints = gameObjects[1]
            pin_joints = gameObjects[2]
            lines = gameObjects[3]
            
            for circle in circles:
                space.add(circle.body,circle.shape)
            for line in lines:
                space.add(line.body,line.shape)
            for join in spring_joints:
                space.add(join.joint)
            for join in pin_joints:
                space.add(join.joint)


        t1 = time.time()

        startTime = pygame.time.get_ticks()
        
        while True:
            currentDistance = max([circle.body.position.x for circle in circles])-600
            # -------------------------------------- GAME ENDS --------------------------------------
            t2 = time.time()
            if(t2-t1 >= self.DURATION):

                self.distance = currentDistance
                self.circle_list = circles
                self.spring_joint_list = spring_joints
                self.pin_joint_list = pin_joints
                self.line_list = lines

                break;
            # ----------------------------------------------------------------------------------------



            for event in pygame.event.get():
                if event.type == QUIT:
                    pygame.quit()
                    sys.exit(0)
                elif event.type == KEYDOWN and event.key == K_ESCAPE:
                    sys.exit(0)

            for joint in spring_joints:
                joint.strain_countdown()

            space.step(1/50.0)

            screen.fill((255,255,255))

            cam_diff = self.updateCamera(circles)
            
            if DRAW_DEBUG:
                space.debug_draw(draw_options)
            else:
                # manual update
                for marker in distance_marker:
                    drawVerticalLine(screen,distance_marker_font,cam_diff,600+marker,color=(0,255,0,0))
                updateLines(screen,lines,cam_diff)
                updateCircles(screen,circles,cam_diff)
                updateJoints(screen,spring_joints,circles,cam_diff,(255,0,0))
                updateJoints(screen,pin_joints,circles,cam_diff,(0,0,255))

            # display timer
            currentTime = pygame.time.get_ticks()
            timer = myfont.render(str((currentTime-startTime)/1000), False, (0, 0, 0))
            screen.blit(timer,(0,0))
            
            #display distance
            distance_display = myfont.render(str(int(currentDistance)), False, (0, 0, 0))
            screen.blit(distance_display,(400,0))
            
            
            pygame.display.flip()
            clock.tick(50)
        
        pygame.quit()
    
        
def add_ball(space):
    mass = 1
    radius = 14
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    x = random.randint(120, 380)
    body.position = x, 550 # 3
    shape = pymunk.Circle(body, radius) # 4
    space.add(body, shape) # 5
    return shape

def add_ball2(space,x,y):
    mass = 1
    radius = 14
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    body.position = x, y # 3
    shape = pymunk.Circle(body, radius) # 4
    space.add(body, shape) # 5
    return body

def draw_ball(screen, ball):
    p = int(ball.body.position.x), 600-int(ball.body.position.y)
    pygame.draw.circle(screen, (0,0,255), p, int(ball.radius), 2)
    
def add_static_L(space,x_body,y_body, length=10000,col=(0,255,0,0)):
    offset = -10000
    return Segment(pymunk.Body.STATIC,space, (offset,100), (length-offset,0), radius=10,color=col)

"""
    body = pymunk.Body(body_type = pymunk.Body.STATIC) # 1
    body.position = (x_body, y_body)
    l1 = pymunk.Segment(body, (0, 0), (length, 0), 5) # 2
    l1.friction = 1
    #l2 = pymunk.Segment(body, (-150, 0), (-150, 50), 5)
    space.add(l1)
    #space.add(l1, l2) # 3
    #return l1,l2
    return l1
    """

def draw_lines(screen, lines):
    for line in lines:
        body = line.body
        pv1 = body.position + line.a.rotated(body.angle) # 1
        pv2 = body.position + line.b.rotated(body.angle)
        p1 = to_pygame(pv1) # 2
        p2 = to_pygame(pv2)
        pygame.draw.lines(screen, THECOLORS["lightgray"], False, [p1,p2])
        
def to_pygame(p):
    """Small hack to convert pymunk to pygame coordinates"""
    return int(p.x), int(-p.y+600)

def add_L(space):
    rotation_center_body = pymunk.Body(body_type = pymunk.Body.STATIC)
    rotation_center_body.position = (300,300)

    rotation_limit_body = pymunk.Body(body_type = pymunk.Body.STATIC) # 1
    rotation_limit_body.position = (200,300)

    body = pymunk.Body(10, 10000)
    body.position = (300,300)
    l1 = pymunk.Segment(body, (-150, 0), (255.0, 0.0), 5.0)
    l2 = pymunk.Segment(body, (-150.0, 0), (-150.0, 50.0), 5.0)

    rotation_center_joint = pymunk.PinJoint(body, rotation_center_body, (0,0), (0,0))
    joint_limit = 25
    rotation_limit_joint = pymunk.SlideJoint(body, rotation_limit_body, (-100,0), (0,0), 0, joint_limit) # 2

    space.add(l1, l2, body, rotation_center_joint, rotation_limit_joint)
    return l1,l2

def add_L2(space):
    rotation_center_body = pymunk.Body(body_type = pymunk.Body.STATIC)
    rotation_center_body.position = (100,500)
    
    circle_body = add_ball2(space,200,400)
    rotation_center_joint = pymunk.PinJoint(circle_body, rotation_center_body, (0,0), (0,0))
    space.add(rotation_center_joint)
    
    circle_body2 = add_ball2(space,400,300)
    
    rotation_center_joint = pymunk.PinJoint(circle_body, circle_body2, (0,0), (0,0))
    space.add(rotation_center_joint)
    
def add_L3(space):
    
    global CAMERA_X
    global CAMERA_Y
    
    b0 = space.static_body
    p = (CAMERA_X,CAMERA_Y)
    v = (50,0)
    arm = Segment(pymunk.Body.DYNAMIC,space,p,v)

    #PivotJoint(space,b0, arm.body, (500,200))
    SimpleMotor(space,b0, arm.body, 5)
    
    return arm

    

class Segment:
    def __init__(self,b_type,space, p0, v, radius=10,color=(0, 255, 0, 0)):
        self.body = pymunk.Body(body_type=b_type)
        self.body.position = p0
        
        self.shape = pymunk.Segment(self.body, (0, 0), v, radius)
        self.shape.density = 0.1
        self.shape.elasticity = 0.5
        self.shape.friction = 1
        self.shape.mass = 10
        #self.shape.filter = pymunk.ShapeFilter(group=1)
        self.shape.color = color
           
        self.b_type_copy = b_type
        self.p0_copy = p0
        self.v_copy = v
        self.radius_copy = radius
        
        if space is not None:
            space.add(self.body, self.shape)
    
    def restoreInitialState(self):
        return Segment(self.b_type_copy,None,self.p0_copy,self.v_copy,self.radius_copy)
        
class Circle:
    def __init__(self, space,pos, radius=20, mass = 1, friction = 0.9):
        
        self.body = pymunk.Body(mass, pymunk.moment_for_circle(mass, 0, radius))
        self.body.position = pos
        
        self.shape = pymunk.Circle(self.body, radius)
        self.shape.density = 0.01
        self.shape.friction = friction
        self.shape.elasticity = 1
        
        self.shape.color = (150, 200, 40, 0)
        
        self.pos_copy = pos
        self.radius_copy = radius
        self.mass_copy = mass
        self.friction_copy = friction
        
        if space is not None:
            space.add(self.body, self.shape)
    
    def restoreInitialState(self):
        return Circle(None,self.pos_copy,self.radius_copy,self.mass_copy,self.friction_copy)
        
class Box:
    def __init__(self, space,p0=(0, 0), p1=(10, 10), d=4):
        x0, y0 = p0
        x1, y1 = p1
        ps = [(x0, y0), (x1, y0), (x1, y1), (x0, y1)]
        for i in range(4):
            segment = pymunk.Segment(b0, ps[i], ps[(i+1) % 4], d)
            segment.elasticity = 1
            segment.friction = 1
            space.add(segment)
    
class PivotJoint:
    def __init__(self, space,b, b2, a=(0, 0), a2=(0, 0), collide=True):
        joint = pymunk.PinJoint(b, b2, a, a2)
        joint.collide_bodies = collide
        space.add(joint)

class SimpleMotor:
    def __init__(self, space,b, b2, rate):
        joint = pymunk.SimpleMotor(b, b2, rate)
        space.add(joint)

class PinJoint:
    def __init__(self, space,b, b2, a=(0, 0), a2=(0, 0),i1=-1,i2=-1):
        self.joint = pymunk.PinJoint(b, b2, a, a2)
        self.i1 = i1
        self.i2 = i2
        
        if space is not None:
            space.add(self.joint)
    
    
    def restoreInitialState(self, f1,f2):
        return PinJoint(None,f1.body, f2.body,i1=self.i1,i2=self.i2)
    

class SpringJoint:
    def __init__(self, space,b, b2, rest_length,stiffness,dampness ,relaxed_duration,strained_duration,a=(0, 0), a2=(0, 0), i1=-1,i2=-1):        
        self.joint = pymunk.DampedSpring(b, b2, a, a2, rest_length,stiffness, dampness)
        self.relaxed_duration = relaxed_duration
        self.strained_duration = strained_duration
        self.state = "relaxed"
        self.current_tick = 0
        self.init_rest_length = rest_length
        self.init_stiffness = stiffness
        
        self.i1 = i1
        self.i2 = i2
        self.rest_length_copy = rest_length
        self.stiffness_copy = stiffness
        self.dampness_copy = dampness
        self.relaxed_duration_copy = relaxed_duration
        self.strained_duration_copy = strained_duration
        self.a_copy = a
        self.a2_copy = a2
        
        if space is not None:
            space.add(self.joint)
    
    def restoreInitialState(self, f1,f2):
        return SpringJoint(None,f1.body, f2.body, self.rest_length_copy,self.stiffness_copy,self.dampness_copy ,self.relaxed_duration_copy,self.strained_duration_copy,self.a_copy, self.a2_copy, self.i1,self.i2)
    
    def strain_countdown(self):
        #print(str(self.current_tick),", ",self.state)
        if self.state == "relaxed":
            if self.relaxed_duration == self.current_tick:
                self.current_tick = 0
                self.state = "strained"
                self.joint._set_rest_length(1)
                #self.joint._set_rest_length(10*self.init_rest_length)
                #self.joint._set_stiffness(self.init_stiffness/10)
        else:
            if self.strained_duration == self.current_tick:
                self.current_tick = 0
                self.state = "relaxed"
                self.joint._set_rest_length(self.init_rest_length) 
                #self.joint._set_rest_length(self.init_rest_length/10)
                #self.joint._set_stiffness(10*self.init_stiffness
        self.current_tick += 1
            
def drawColoredBridge(screen):
    return None
    

def updateLines(screen,lines,camera):
    
    for line in lines:
        shape = line.shape
        body = line.body
        new_body_pos = body.position - camera
        
        pv1 = new_body_pos + shape.a.rotated(body.angle)
        pv2 = new_body_pos + shape.b.rotated(body.angle)
        
        p1 = pv1.x, -pv1.y+600
        p2 = pv2.x, -pv2.y+600
        
        #print(pv1,pv2)
        #print(p1,p2)
        #print(camera)
        #print("-----------")
        pygame.draw.lines(screen, shape.color, False, [p1,p2], int(2*shape.radius))
    #print("----FINISH--")
    
def drawVerticalLine(screen,font,camera,lineDistance,color=(0,255,0,0)):
    
    p1 = Vec2d(lineDistance,100) - camera
    p2 = Vec2d(lineDistance,300) - camera
    
    p1 = Vec2d(p1.x,600-p1.y)
    p2 = Vec2d(p2.x,600-p2.y)
    
    draw_x = p1.x
    
    #print(p1,p2,"It should be visible on screen")
    if draw_x >= 0 and draw_x <= 600:
        pygame.draw.lines(screen, color, False, [p1,p2], 5)
        distance_marker_display = font.render(str(lineDistance-600), False, (0, 0, 0))
        screen.blit(distance_marker_display,p2)
    
def updateJoints(screen,joints,circles,camera,color):
    
    for joint in joints:
        c1 = circles[joint.i1]
        c2 = circles[joint.i2]
        
        #print("update Joints: ",joint.i1,joint.i2)
        
        c1_pos = c1.body.position - camera
        c2_pos = c2.body.position - camera
    
        p1 = c1_pos.x, -c1_pos.y+600
        p2 = c2_pos.x, -c2_pos.y+600
        
        #print("update Joints: ",p1,p2)
        
        pygame.draw.lines(screen, color, False, [p1,p2], 5)
    
def updateCircles(screen,circles,camera):
    
    for circle in circles:
        #print("update Circles: ",circle.body.position)
        
        shape = circle.shape
        body = circle.body
        v = body.position
        r = shape.radius
        rot = body.rotation_vector
        
        new_body_pos = body.position - camera
        p2 = Vec2d(rot.x, rot.y) * r * 0.9
        
        # width is 2, radius should always be greater than width
        pygame.draw.circle(screen, shape.color, (int(new_body_pos.x), int(-new_body_pos.y+600)), int(r), 2)
        
        startPoint = new_body_pos
        endPoint = new_body_pos+p2
        pygame.draw.line(screen, (255,0,0), Vec2d(startPoint.x,-startPoint.y+600), Vec2d(endPoint.x,-endPoint.y+600))
        
        #print("update Circles: ",new_body_pos)
        #print(pv1,pv2)
        #print(p1,p2)
        #print(camera)
        #print("-----------")
    #print("----FINISH--")
    
def createMachine(space):

    global CAMERA_X
    global CAMERA_Y
    
    b0 = space.static_body
    p = (CAMERA_X-100,CAMERA_Y)
    v = (50,0)
    arm = Segment(pymunk.Body.DYNAMIC,space,p,v)
    circle = Circle(space,(CAMERA_X-100,CAMERA_Y+50))
    
    
    PinJoint(space,circle.body, arm.body)
    SimpleMotor(space,b0, arm.body, 5)
    
    return arm,circle

def createMachine2(space):

    circle = Circle(space,(300,400))
    circle2 = Circle(space,(400,400))
    
    joint = pymunk.DampedSpring(circle2.body, circle.body, (0,0), (0, 0), 10, 50, 0)
    space.add(joint)
    
    return circle,circle2,joint

def createRandomMachine(space, PRINT_DEBUG):
    
    MIN_CIRCLES = 6
    MAX_CIRCLES = 6
    
    # MIN always greater than 1 to avoid errors (wider than radius when drawin)
    MIN_RADIUS = 20
    MAX_RADIUS = 20
    
    MIN_MASS = 20 
    MAX_MASS = 100
    
    MIN_FRICTION = 1
    MAX_FRICTION = 10
    
    X_MIN = 0
    X_MAX = 600
    Y_MIN = 200
    Y_MAX = 400
    
    SPRING_JOINT_PROB = 0.5
    PIN_JOINT_PROB = 0.5
    
    MIN_REST_LENGTH = 200
    MAX_REST_LENGTH = 200
    
    MIN_STIFFNESS = 200
    MAX_STIFFNESS = 200
    
    MIN_DAMPNESS = 50
    MAX_DAMPNESS = 50
    
    MIN_RELAXED_DURATION = 50*0.1
    MAX_RELAXED_DURATION = 50*3
    
    MIN_STRAINED_DURATION = 50*0.1
    MAX_STRAINED_DURATION = 50*3
    
    numberOfCircles = random.randint(MIN_CIRCLES,MAX_CIRCLES)
    if PRINT_DEBUG:
        print("Number of Circles: ", numberOfCircles)
    radius_list = [random.randint(MIN_RADIUS,MAX_RADIUS) for x in range(numberOfCircles)]
    if PRINT_DEBUG:
        print("Radius List: ", radius_list)
    mass_list = [MIN_MASS + int((MAX_MASS-MIN_MASS)*random.random()) for x in range(numberOfCircles)]
    if PRINT_DEBUG:
        print("Mass List: ", mass_list)
    friction_list = [MIN_FRICTION + int((MAX_FRICTION-MIN_FRICTION)*random.random()) for x in range(numberOfCircles)]
    if PRINT_DEBUG:
        print("Friction List: ", friction_list)
    position_list = [Vec2d(X_MIN + (X_MAX-X_MIN) * random.random(), Y_MIN + (Y_MAX-Y_MIN) * random.random()) for x in range(numberOfCircles)]
    if PRINT_DEBUG:    
        print("Position List: ", position_list)
    
    circles = [Circle(space,position_list[i],radius_list[i],mass_list[i],friction_list[i]) for i in range(numberOfCircles)]
    
    nodes = [i for i in range(numberOfCircles)]
    while True:
        edges = []
        
        rest_length_list = []
        stiffness_list = []
        dampness_list = []
        relaxed_duration_list = []
        strained_duration_list = []

        spring_joints = []
        pin_joints = []

        for pair in itertools.combinations(circles, r=2):

            c1 = pair[0]

            c2 = pair[1]

            if random.random() < SPRING_JOINT_PROB:
                rest_length = MIN_REST_LENGTH + (MAX_REST_LENGTH-MIN_REST_LENGTH)*random.random()
                stiffness = MIN_STIFFNESS + (MAX_STIFFNESS-MIN_STIFFNESS)*random.random()
                dampness = MIN_DAMPNESS + (MAX_DAMPNESS-MIN_DAMPNESS)*random.random()
                relaxed_duration = MIN_RELAXED_DURATION + int((MAX_RELAXED_DURATION-MIN_RELAXED_DURATION)*random.random())
                strained_duration = MIN_STRAINED_DURATION + int((MAX_STRAINED_DURATION-MIN_STRAINED_DURATION)*random.random())

                rest_length_list.append(rest_length)
                stiffness_list.append(stiffness)
                dampness_list.append(dampness)
                relaxed_duration_list.append(relaxed_duration)
                strained_duration_list.append(strained_duration)

                spring_joints.append(SpringJoint(space,c1.body, c2.body, rest_length, stiffness,dampness, relaxed_duration,strained_duration,i1=circles.index(c1),i2=circles.index(c2)))
                
            elif random.random() < PIN_JOINT_PROB:
                pin_joints.append(PinJoint(space,c1.body, c2.body,i1=circles.index(c1),i2=circles.index(c2)))
                
        edges.extend([[joint.i1,joint.i2] for joint in pin_joints])
        edges.extend([[joint.i1,joint.i2] for joint in spring_joints])
        
        if Graph(nodes,edges).isStronglyConnected():
            break;

                
    if PRINT_DEBUG:
        print("Number of joints: ", len(joints))
        print("Rest_lengths of joints: ", rest_length_list)
        print("Stiffness of joints: ", stiffness_list)
        print("Dampness of joints: ", dampness_list)
        print("Relaxed duration of joints: ", relaxed_duration_list)
        print("Strained duration of joints: ", strained_duration_list)
    
    return circles,spring_joints,pin_joints


def createMachine3(space):

    global CAMERA_X
    global CAMERA_Y
    
    b0 = space.static_body
    
    p1 = Vec2d(CAMERA_X-100,CAMERA_Y+200)
    p2 = Vec2d(CAMERA_X-100,CAMERA_Y+30)
    
    v1 = Vec2d(50,0)
    v2 = Vec2d(30,0)
    v3 = Vec2d(0,30)
    
    k = Vec2d(40,0)

    arm = Segment(pymunk.Body.DYNAMIC,space,p1,v1)
    arm2 = Segment(pymunk.Body.DYNAMIC,space,p1+v1+k,v2)
    
    joint = pymunk.DampedSpring(b0, arm.body, p1, (0, 0), 10, 10, 0)
    #joint._set_rest_length(1)
    space.add(joint)
    
    """
    
    PivotJoint(space,arm2.body, arm.body, (0,0),v1)
    #print(arm2.body.position,arm.body.position+v1)
    SimpleMotor(space,b0, arm.body, -2)
    SimpleMotor(space,b0, arm2.body, 4)
"""
    return arm,arm2,joint

class Graph:
    
    def __init__(self, nodes, edges):
        self.nodes = nodes
        self.edges = edges
    
    def isStronglyConnected(self):
        for node1 in self.nodes:
            for node2 in self.nodes:
                if not node1 == node2:
                    if not self.DFS(node1,node2):
                        return False
        return True
        
    def DFS(self,start,goal,randomized=False):
        frontier = [start]
        explored = []

        while not frontier==[]:
            next_node = frontier.pop(0)
            explored.append(next_node)
            if next_node == goal:
                return True
            neighbours = self.getNeighbours(next_node)
            search = self.extractAlreadySearched(neighbours,frontier, explored)
            if randomized:
                random.shuffle(search)
            frontier = search + frontier
        return False
        
    def getNeighbours(self,node):
        neighbours = []
        for edge in self.edges:
            if edge[0] == node:
                neighbours.append(edge[1])
            elif edge[1] == node:
                neighbours.append(edge[0])
            else:
                continue
            
        return neighbours

    def extractAlreadySearched(self,neighbours, explored, frontier):
        search = []
        for neighbour in neighbours:
            if neighbour not in explored and neighbour not in frontier:
                search.append(neighbour)
        return search
    
    
    
    
#--------------------START-------------------------------------------

distances = []
objects = []
MAX_ITER = 1000
game = Game()

x_data = [i for i in range(MAX_ITER+1)]
y_data = [0]
t1 = time.time()
for x in range(MAX_ITER):
    print(x+1,". Child is simulated.")
    game.simulatePhysics()
    y_data.append(max(game.distance,y_data[-1]))
    distances.append(game.distance)
    objects.append(game.getGameObjects())
t2 = time.time()
#print(str(t2-t1))
best_machine = np.argmax(np.array(distances))
cop = copy.deepcopy(objects[best_machine])
game.start(True,objects[best_machine],True)
game.start(True,cop,False)
#game.start(DRAW_DEBUG=True)
print(game.distance)

plt.plot(x_data,y_data)
plt.show()

print(b)

1 . Child is simulated.
2 . Child is simulated.
3 . Child is simulated.
4 . Child is simulated.
5 . Child is simulated.
6 . Child is simulated.
7 . Child is simulated.
8 . Child is simulated.
9 . Child is simulated.
10 . Child is simulated.
11 . Child is simulated.
12 . Child is simulated.
13 . Child is simulated.
14 . Child is simulated.
15 . Child is simulated.
16 . Child is simulated.
17 . Child is simulated.
18 . Child is simulated.
19 . Child is simulated.
20 . Child is simulated.
21 . Child is simulated.
22 . Child is simulated.
23 . Child is simulated.
24 . Child is simulated.
25 . Child is simulated.
26 . Child is simulated.
27 . Child is simulated.
28 . Child is simulated.
29 . Child is simulated.
30 . Child is simulated.
31 . Child is simulated.
32 . Child is simulated.
33 . Child is simulated.
34 . Child is simulated.
35 . Child is simulated.
36 . Child is simulated.
37 . Child is simulated.
38 . Child is simulated.
39 . Child is simulated.
40 . Child is simulated.
41 . Chil

330 . Child is simulated.
331 . Child is simulated.
332 . Child is simulated.
333 . Child is simulated.
334 . Child is simulated.
335 . Child is simulated.
336 . Child is simulated.
337 . Child is simulated.
338 . Child is simulated.
339 . Child is simulated.
340 . Child is simulated.
341 . Child is simulated.
342 . Child is simulated.
343 . Child is simulated.
344 . Child is simulated.
345 . Child is simulated.
346 . Child is simulated.
347 . Child is simulated.
348 . Child is simulated.
349 . Child is simulated.
350 . Child is simulated.
351 . Child is simulated.
352 . Child is simulated.
353 . Child is simulated.
354 . Child is simulated.
355 . Child is simulated.
356 . Child is simulated.
357 . Child is simulated.
358 . Child is simulated.
359 . Child is simulated.
360 . Child is simulated.
361 . Child is simulated.
362 . Child is simulated.
363 . Child is simulated.
364 . Child is simulated.
365 . Child is simulated.
366 . Child is simulated.
367 . Child is simulated.
368 . Child 

655 . Child is simulated.
656 . Child is simulated.
657 . Child is simulated.
658 . Child is simulated.
659 . Child is simulated.
660 . Child is simulated.
661 . Child is simulated.
662 . Child is simulated.
663 . Child is simulated.
664 . Child is simulated.
665 . Child is simulated.
666 . Child is simulated.
667 . Child is simulated.
668 . Child is simulated.
669 . Child is simulated.
670 . Child is simulated.
671 . Child is simulated.
672 . Child is simulated.
673 . Child is simulated.
674 . Child is simulated.
675 . Child is simulated.
676 . Child is simulated.
677 . Child is simulated.
678 . Child is simulated.
679 . Child is simulated.
680 . Child is simulated.
681 . Child is simulated.
682 . Child is simulated.
683 . Child is simulated.
684 . Child is simulated.
685 . Child is simulated.
686 . Child is simulated.
687 . Child is simulated.
688 . Child is simulated.
689 . Child is simulated.
690 . Child is simulated.
691 . Child is simulated.
692 . Child is simulated.
693 . Child 

984 . Child is simulated.
985 . Child is simulated.
986 . Child is simulated.
987 . Child is simulated.
988 . Child is simulated.
989 . Child is simulated.
990 . Child is simulated.
991 . Child is simulated.
992 . Child is simulated.
993 . Child is simulated.
994 . Child is simulated.
995 . Child is simulated.
996 . Child is simulated.
997 . Child is simulated.
998 . Child is simulated.
999 . Child is simulated.
1000 . Child is simulated.


SystemExit: 0